In [ ]:
import openpyxl

# Открываем Excel файл
wb = openpyxl.load_workbook('aav.xlsx', data_only=False)

# Список названий листов
sheet_names = wb.sheetnames

# Проходим по каждому листу
for sheet_name in sheet_names:
    sheet = wb[sheet_name]
    print(f'Лист: {sheet_name}')
    
    # Проходим по всем ячейкам листа
    for row in sheet.iter_rows():
        for cell in row:
            # Если в ячейке есть формула, выводим её
            if isinstance(cell.value, str) and cell.value.startswith('='):
                print(f'{cell.coordinate}: {cell.value}')
            # Если в ячейке текст, выводим его
            elif isinstance(cell.value, str) and not cell.value.startswith('='):
                print(f'{cell.coordinate}: {cell.value}')
            # Если в ячейке числовое значение, выводим его
            elif cell.value is not None:
                print(f'{cell.coordinate}: {cell.value}')


Лист: Sheet1
A2: перевод единиц объема наработки в объем КЖ 
H2: Расчет объема КЖ к плазамидам и PEI 10%
A3: Единица объема 
B3: Т175 
C3: КРУГЛЫЕ 10см
D3: 5-level cellstack
G3: Единица объема 
H3: Т175 
I3: КРУГЛЫЕ 10см
J3: 5-level cellstack
A4: Объем среды, мл
B4: 30
C4: ND
D4: ND
G4: Объем среды, мл
H4: 30
I4: 8
J4: calc
A5: Площадь поверхности, см2
B5: 175
C5: 100
D5: 3180
G5: Объем ТФ микса на флакон, мл
H5: 3
A6: Коэффициент по площади относительно Т175 
B6: 1
C6: =C5/B5
D6: =D5/B5
G6: Объем ТФ микса (DMEM+mix), мл
H6: =H5*B6
I6: =H5*C6
J6: =D6*H5
A8: Название плазмиды 
B8: Длина плазмиды, bp
C8: Кол-во мкг на 1 pmol
D8: Соотношение плазмид
E8: Кол-во ДНК с учетом соотношения 
F8: Днк всего 
G8: Объем наработки
H8: Коэффициент перевода площадь-КЖ
I8: Кол-во ДНК на 1 мл КЖ
J8: Суммарное кол-во плазмид
K8: Соотношение PEI/DNA
L8: Суммарное кол-во PEI, мкл 
M8: Коэффициент
N8: Суммарное кол-во плазмид
O8: Концентрация плазмид, мкг/мкл
P8: Объем плазмид, мкл
Q8: Общий объем пламид, м

In [ ]:
import numpy as np

# Функция для расчета
def calculate_transfection(
    plasmid_helper_concentration,  # Концентрация pAAV-Helper (мкг/пикомоль)
    plasmid_capsid_concentration,  # Концентрация капсида (мкг/пикомоль)
    plasmid_transgene_concentration,  # Концентрация трансгена (мкг/пикомоль)
    plasmid_ratio,  # Соотношение плазмид (pHelper : Capsid : Transgene)
    plasmid_helper_length,  # Длина pAAV-Helper (bp)
    plasmid_capsid_length,  # Длина капсида (bp)
    plasmid_transgene_length,  # Длина трансгена (bp)
    volume_type,  # Тип объема наработки
    volume_amount,  # Количество объемов наработки
    volume_work_ml  # Объем наработки (мл)
):
    # Парсим соотношение плазмид
    plasmid_ratio = list(map(float, plasmid_ratio.split(':')))
    
    # Проверка на корректность значений
    if (np.isnan(plasmid_helper_concentration) or np.isnan(plasmid_capsid_concentration) or 
        np.isnan(plasmid_transgene_concentration) or len(plasmid_ratio) != 3 or 
        np.isnan(plasmid_helper_length) or np.isnan(plasmid_capsid_length) or 
        np.isnan(plasmid_transgene_length)):
        raise ValueError("Пожалуйста, убедитесь, что все поля заполнены корректно.")
    
    # Устанавливаем коэффициент перевода в зависимости от типа емкости
    if volume_type == "T175":
        coef = 1
    elif volume_type == "КРУГЛЫЕ 10см":
        coef = 0.6
    elif volume_type == "5-level cellstack":
        coef = 18
    else:
        raise ValueError("Неверный тип объема наработки.")
    
    # Проверка на корректность значений объема
    if np.isnan(volume_work_ml) or np.isnan(coef) or np.isnan(volume_amount):
        raise ValueError("Пожалуйста, убедитесь, что объем и коэффициент перевода введены корректно.")
    
    # Расчет объема КЖ (ml)
    KJ = volume_work_ml * coef * volume_amount
    
    # Расчет суммарного количества ДНК для всех трех плазмид с учетом их соотношений
    total_DNA = (plasmid_helper_concentration * plasmid_ratio[0]) + \
                (plasmid_capsid_concentration * plasmid_ratio[1]) + \
                (plasmid_transgene_concentration * plasmid_ratio[2])
    
    # Расчет общего количества вирусных геномов (VG) и объема микса
    VG = total_DNA * 1e6  # Примерная формула для количества вирусных геномов (VG)
    mix_volume = VG / (10 * 1e6)  # Примерная формула для объема микса (ml)
    
    # Возвращаем результаты
    return {
        "Объем КЖ (мл)": KJ,
        "Общее количество ДНК (нг)": total_DNA,
        "Общее количество вирусных геномов (VG)": VG,
        "Объем микса (мл)": mix_volume,
        "Длина pAAV-Helper (bp)": plasmid_helper_length,
        "Длина капсида (bp)": plasmid_capsid_length,
        "Длина трансгена (bp)": plasmid_transgene_length
    }

# Пример использования функции
results = calculate_transfection(
    plasmid_helper_concentration=1.1,
    plasmid_capsid_concentration=2.335,
    plasmid_transgene_concentration=1.01,
    plasmid_ratio="1:1:1",
    plasmid_helper_length=11635,
    plasmid_capsid_length=8369,
    plasmid_transgene_length=7285,
    volume_type="T175",
    volume_amount=8,
    volume_work_ml=30
)

# Вывод результатов
for key, value in results.items():
    print(f"{key}: {value}")


Объем КЖ (мл): 240
Общее количество ДНК (нг): 4.445
Общее количество вирусных геномов (VG): 4445000.0
Объем микса (мл): 0.4445
Длина pAAV-Helper (bp): 11635
Длина капсида (bp): 8369
Длина трансгена (bp): 7285
